In [12]:
import numpy as np
import pandas as pd
import pathlib
from tqdm import tqdm
import F5signal as f5s
from models.research.audioset.vggish import vggish_params
from models.research.audioset.vggish import vggish_slim
from models.research.audioset.vggish import vggish_input
from models.research.audioset.vggish import vggish_postprocess

In [13]:
def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)

  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }

In [14]:
def ProcessWithVGGish(vgg, x, sr):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a whitened version of the embeddings. Sound must be scaled to be
  floats between -1 and +1.'''

  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.waveform_to_examples(x, sr)
  # print('Log Mel Spectrogram example: ', input_batch[0])

  [embedding_batch] = sess.run([vgg['embedding']],
                               feed_dict={vgg['features']: input_batch})

  # Postprocess the results to produce whitened quantized embeddings.
  pca_params_path = 'vggish_pca_params.npz'

  pproc = vggish_postprocess.Postprocessor(pca_params_path)
  postprocessed_batch = pproc.postprocess(embedding_batch)
  # print('Postprocessed VGGish embedding: ', postprocessed_batch[0])
  return postprocessed_batch[0]

In [15]:
def EmbeddingsFromVGGish(vgg, x, sr):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a dictionary of embeddings from the different layers
  of the model.'''
  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.waveform_to_examples(x, sr)
  # print('Log Mel Spectrogram example: ', input_batch[0])

  layer_names = vgg['layers'].keys()
  tensors = [vgg['layers'][k] for k in layer_names]
  
  results = sess.run(tensors,
                     feed_dict={vgg['features']: input_batch})

  resdict = {}
  for i, k in enumerate(layer_names):
    resdict[k] = results[i]
    
  return resdict

In [16]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork(0.01)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [17]:
files = pathlib.Path(r'C:\Users\markh\Work\my_dcase\testing\dev_data\fan\test')
files = files.glob('*.wav')
files=list(files)

In [18]:
df = pd.DataFrame(columns=['name'] + ['y'] + ['id'] + list(range(0, 128)))

In [19]:
df

,name,y,id,0,1,2,3,4,5,6,...,118,119,120,121,122,123,124,125,126,127


In [20]:
for num,wav in tqdm(enumerate(files),total=len(files)):
    name=wav.stem.split('_')[0]
    id_wav=int(wav.stem.split('_')[2])
    num_id_wav=int(wav.stem.split('_')[3])
    wave = f5s.read_wave(str(wav))
    x, sr = wave.ys, wave.framerate
    postprocessed_batch = ProcessWithVGGish(vgg, x, sr)
    if name=='normal':
        l = [str(wav), 0, id_wav]
        l.extend(postprocessed_batch)
        df.loc[num] = l
    else:
        l = [str(wav), 1, id_wav]
        l.extend(postprocessed_batch)
        df.loc[num] = l


 16%|█▋        | 309/1875 [50:30<4:15:57,  9.81s/it]


 32%|███▏      | 608/1875 [1:48:14<3:45:33, 10.68s/it]


 48%|████▊     | 904/1875 [2:43:55<2:56:04, 10.88s/it]


 56%|█████▌    | 1052/1875 [3:11:41<2:29:57, 10.93s/it]


 72%|███████▏  | 1344/1875 [4:01:23<1:35:22, 10.78s/it]


...


100%|█████████▉| 1873/1875 [5:27:19<00:20, 10.49s/it]

['C:\\Users\\markh\\Work\\my_dcase\\testing\\dev_data\\fan\\test\\normal_id_06_00000098.wav', 0, 6, 164, 42, 149, 35, 255, 92, 151, 99, 72, 218, 189, 53, 12, 241, 153, 99, 8, 186, 152, 116, 83, 255, 103, 12, 25, 255, 132, 255, 133, 107, 170, 100, 107, 153, 99, 234, 173, 67, 181, 70, 111, 181, 0, 141, 42, 194, 109, 17, 183, 157, 242, 72, 255, 109, 255, 129, 252, 134, 186, 245, 178, 111, 86, 233, 203, 108, 240, 83, 65, 0, 0, 255, 255, 77, 0, 0, 4, 40, 156, 255, 36, 255, 94, 233, 0, 0, 241, 175, 162, 190, 0, 238, 255, 255, 143, 255, 21, 0, 143, 131, 0, 29, 0, 0, 0, 39, 48, 255, 255, 255, 180, 8, 0, 0, 0, 0, 255, 255, 20, 139, 174, 120, 53, 224, 252, 0, 0, 255]



100%|█████████▉| 1874/1875 [5:27:29<00:10, 10.49s/it]

['C:\\Users\\markh\\Work\\my_dcase\\testing\\dev_data\\fan\\test\\normal_id_06_00000099.wav', 0, 6, 158, 29, 135, 28, 255, 110, 143, 128, 79, 219, 160, 75, 0, 255, 152, 117, 76, 151, 122, 121, 102, 255, 86, 45, 51, 255, 125, 255, 85, 79, 177, 31, 134, 113, 81, 153, 152, 149, 104, 0, 181, 223, 0, 214, 33, 234, 255, 68, 105, 147, 171, 54, 255, 78, 255, 143, 193, 75, 169, 198, 138, 141, 198, 255, 255, 48, 255, 75, 113, 72, 38, 255, 250, 0, 0, 0, 0, 201, 177, 255, 0, 255, 98, 167, 3, 0, 230, 255, 68, 255, 0, 255, 157, 213, 182, 255, 0, 0, 255, 88, 68, 0, 0, 0, 0, 109, 56, 255, 179, 255, 169, 47, 0, 0, 0, 0, 255, 255, 0, 211, 180, 112, 168, 255, 255, 0, 0, 255]



100%|██████████| 1875/1875 [5:27:39<00:00, 10.48s/it]


In [21]:
df.to_csv('tmp.csv')

In [ ]:
df.